In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys


sys.path.insert(0, "..")   

from src.ML_benchmark_prediction_analysis import (
    metrics_per_category
)

In [16]:
%reload_ext autoreload
%autoreload 2

In [17]:
path = Path("../output/data/gtdb_causal_onco_tsg_gene_disease_icd10_protein_class.csv")
df = pd.read_csv(path)
df.head()

,Gene,Role,ICD10_Code,UMLS,Disease_Type,Association_Type,Disease_Name,Entry,Reviewed,Entry Name_x,...,Level_1,Level_2,Level_3,Level_4,Level_5,Level_6,Level_7,accession,sequence,organism
0,SH2B3,TSG,R10,C0000737,phenotype,CausalMutation,Abdominal Pain,Q9UQQ2,reviewed,SH2B3_HUMAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,STK11,TSG,R10,C0000737,phenotype,CausalMutation,Abdominal Pain,Q15831,reviewed,STK11_HUMAN,...,Protein class,Enzyme,Kinase,Protein Kinase,CAMK protein kinase group,CAMK protein kinase CAMK1 family,CAMK protein kinase LKB subfamily,Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...,Homo sapiens
2,STK11,TSG,R10,C0000737,phenotype,CausalMutation,Abdominal Pain,Q15831,reviewed,STK11_HUMAN,...,Protein class,Enzyme,Kinase,Protein Kinase,CAMK protein kinase group,CAMK protein kinase CAMK1 family,CAMK protein kinase LKB subfamily,Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...,Homo sapiens
3,STK11,TSG,R10,C0000737,phenotype,CausalMutation,Abdominal Pain,Q15831,reviewed,STK11_HUMAN,...,Protein class,Enzyme,Kinase,Protein Kinase,CAMK protein kinase group,CAMK protein kinase CAMK1 family,CAMK protein kinase LKB subfamily,Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...,Homo sapiens
4,STK11,TSG,R10,C0000737,phenotype,CausalMutation,Abdominal Pain,Q15831,reviewed,STK11_HUMAN,...,Protein class,Enzyme,Kinase,Protein Kinase,CAMK protein kinase group,CAMK protein kinase CAMK1 family,CAMK protein kinase LKB subfamily,Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...,Homo sapiens


In [21]:
df.columns

Index(['Gene', 'Role', 'ICD10_Code', 'UMLS', 'Disease_Type',
       'Association_Type', 'Disease_Name', 'Entry', 'Reviewed', 'Entry Name_x',
       'Protein names', 'Gene Names_x', 'Organism', 'Length', 'Sequence',
       'EC number', 'Active site', 'Binding site', 'Rhea ID', 'Pathway',
       'Catalytic activity', 'Tissue specificity', 'Uniprot', 'Gene_x',
       'Ensembl', 'Target Class', 'FDA Approved', 'Entry Name_y',
       'Gene Names_y', 'Gene_y', 'component_id', 'chembl_id',
       'target_pref_name', 'protein_class_name', 'class_level', 'Level_1',
       'Level_2', 'Level_3', 'Level_4', 'Level_5', 'Level_6', 'Level_7',
       'accession', 'sequence', 'organism'],
      dtype='object')

In [22]:
df["Organism"].unique()

array(['Homo sapiens (Human)', nan], dtype=object)

In [30]:
mask = df["Organism"].notna() & df["Organism"].str.contains("Homo sapiens", regex=False)
df[mask]


,Gene,Role,ICD10_Code,UMLS,Disease_Type,Association_Type,Disease_Name,Entry,Reviewed,Entry Name_x,...,Level_1,Level_2,Level_3,Level_4,Level_5,Level_6,Level_7,accession,sequence,organism
0,SH2B3,TSG,R10,C0000737,phenotype,CausalMutation,Abdominal Pain,Q9UQQ2,reviewed,SH2B3_HUMAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,STK11,TSG,R10,C0000737,phenotype,CausalMutation,Abdominal Pain,Q15831,reviewed,STK11_HUMAN,...,Protein class,Enzyme,Kinase,Protein Kinase,CAMK protein kinase group,CAMK protein kinase CAMK1 family,CAMK protein kinase LKB subfamily,Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...,Homo sapiens
2,STK11,TSG,R10,C0000737,phenotype,CausalMutation,Abdominal Pain,Q15831,reviewed,STK11_HUMAN,...,Protein class,Enzyme,Kinase,Protein Kinase,CAMK protein kinase group,CAMK protein kinase CAMK1 family,CAMK protein kinase LKB subfamily,Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...,Homo sapiens
3,STK11,TSG,R10,C0000737,phenotype,CausalMutation,Abdominal Pain,Q15831,reviewed,STK11_HUMAN,...,Protein class,Enzyme,Kinase,Protein Kinase,CAMK protein kinase group,CAMK protein kinase CAMK1 family,CAMK protein kinase LKB subfamily,Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...,Homo sapiens
4,STK11,TSG,R10,C0000737,phenotype,CausalMutation,Abdominal Pain,Q15831,reviewed,STK11_HUMAN,...,Protein class,Enzyme,Kinase,Protein Kinase,CAMK protein kinase group,CAMK protein kinase CAMK1 family,CAMK protein kinase LKB subfamily,Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...,Homo sapiens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31084,KIF1A,TSG,Q66,C4082169,disease,CausalMutation,Metatarsus Varus,Q12756,reviewed,KIF1A_HUMAN,...,Protein class,Other cytosolic protein,NaN,NaN,NaN,NaN,NaN,Q12756,MAGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTIVNPKQPKET...,Homo sapiens
31085,KIF1A,TSG,Q66,C4082169,disease,CausalMutation,Metatarsus Varus,Q12756,reviewed,KIF1A_HUMAN,...,Protein class,Other cytosolic protein,NaN,NaN,NaN,NaN,NaN,Q12756,MAGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTIVNPKQPKET...,Homo sapiens
31086,KIF1A,TSG,Q66,C4082169,disease,CausalMutation,Metatarsus Varus,Q12756,reviewed,KIF1A_HUMAN,...,Protein class,Other cytosolic protein,NaN,NaN,NaN,NaN,NaN,Q12756,MAGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTIVNPKQPKET...,Homo sapiens
31087,COL2A1,TSG,Q77,C4520892,disease,CausalMutation,Otospondylomegaepiphyseal dysplasia,P02458,reviewed,CO2A1_HUMAN,...,Protein class,Structural protein,NaN,NaN,NaN,NaN,NaN,P02458,MIRLGAPQTLVLLTLLVAAVLRCQGQDVQEAGSCVQDGQRYNDKDV...,Homo sapiens


In [24]:
df["Tissue specificity"].unique()

array(['TISSUE SPECIFICITY: Preferentially expressed by lymphoid cell lines.',
       'TISSUE SPECIFICITY: Ubiquitously expressed. Strongest expression in testis and fetal liver.',
       'TISSUE SPECIFICITY: Expressed in the brain, heart, liver, skeletal muscle, pancreas and placenta. Isoform 1 is predominant in the brain and spinal cord. Isoform 4 is more abundant in the cerebellum. In the brain, broadly expressed in the amygdala, caudate nucleus, corpus callosum, hippocampus, hypothalamus, substantia nigra, subthalamic nucleus and thalamus. {ECO:0000269|PubMed:8896555, ECO:0000269|PubMed:8896556, ECO:0000269|PubMed:8896557, ECO:0000269|PubMed:9480749}.',
       'TISSUE SPECIFICITY: Detected in brain, peripheral nerve, lung, colon and muscle. {ECO:0000269|PubMed:8417346}.',
       'TISSUE SPECIFICITY: Ubiquitous. Absent in primary spermatocytes. {ECO:0000269|PubMed:9591631}.',
       nan,
       'TISSUE SPECIFICITY: Expressed in most adult tissues, including a variety of hematoipoiet

In [19]:
df[["Gene", "Protein names","chembl_id","sequence", "Uniprot", "Sequence"]]

,Gene,Protein names,chembl_id,sequence,Uniprot,Sequence
0,SH2B3,SH2B adapter protein 3 (Lymphocyte adapter pro...,NaN,NaN,Q9UQQ2,MNGPALQPSSPSSAPSASPAAAPRGWSEFCELHAVAAARELARQYW...
1,STK11,Serine/threonine-protein kinase STK11 (EC 2.7....,CHEMBL5606,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...,Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...
2,STK11,Serine/threonine-protein kinase STK11 (EC 2.7....,CHEMBL3885534,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...,Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...
3,STK11,Serine/threonine-protein kinase STK11 (EC 2.7....,CHEMBL5606,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...,Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...
4,STK11,Serine/threonine-protein kinase STK11 (EC 2.7....,CHEMBL3885534,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...,Q15831,MEVVDPQQLGMFTEGELMSVGMDTFIHRIDSTEVIYQPRRKRAKLI...
...,...,...,...,...,...,...
31084,KIF1A,Kinesin-like protein KIF1A (EC 5.6.1.3) (Axona...,CHEMBL3308914,MAGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTIVNPKQPKET...,Q12756,MAGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTIVNPKQPKET...
31085,KIF1A,Kinesin-like protein KIF1A (EC 5.6.1.3) (Axona...,CHEMBL3308914,MAGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTIVNPKQPKET...,Q12756,MAGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTIVNPKQPKET...
31086,KIF1A,Kinesin-like protein KIF1A (EC 5.6.1.3) (Axona...,CHEMBL3308914,MAGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTIVNPKQPKET...,Q12756,MAGASVKVAVRVRPFNSREMSRDSKCIIQMSGSTTTIVNPKQPKET...
31087,COL2A1,Collagen alpha-1(II) chain (Alpha-1 type II co...,CHEMBL2364188,MIRLGAPQTLVLLTLLVAAVLRCQGQDVQEAGSCVQDGQRYNDKDV...,P02458,MIRLGAPQTLVLLTLLVAAVLRCQGQDVQEAGSCVQDGQRYNDKDV...


In [20]:
df["Binding site"]

0                                                      NaN
1        BINDING 55..63; /ligand="ATP"; /ligand_id="ChE...
2        BINDING 55..63; /ligand="ATP"; /ligand_id="ChE...
3        BINDING 55..63; /ligand="ATP"; /ligand_id="ChE...
4        BINDING 55..63; /ligand="ATP"; /ligand_id="ChE...
                               ...                        
31084    BINDING 102; /ligand="ATP"; /ligand_id="ChEBI:...
31085    BINDING 102; /ligand="ATP"; /ligand_id="ChEBI:...
31086    BINDING 102; /ligand="ATP"; /ligand_id="ChEBI:...
31087    BINDING 1301; /ligand="Ca(2+)"; /ligand_id="Ch...
31088    BINDING 1301; /ligand="Ca(2+)"; /ligand_id="Ch...
Name: Binding site, Length: 31089, dtype: object

In [ ]:
path = Path("/sc/arion/projects/DiseaseGeneCell/Huang_lab_project/wangcDrugRepoProject/BindDBdata/All_BindingDB/ESM2_BindingDB_random")
df = pd.read_parquet(path / "train.parquet", engine='fastparquet')
df.head()

In [ ]:
path = "/sc/arion/projects/DiseaseGeneCell/Huang_lab_project/drug_discovery/output/models/Random_Forest_ESMv1_BindingDB_random_predictions.csv"
pred_df = pd.read_csv(Path(path))
pred_df.rename(columns={"drug_smile":"Drug", "affinity": "affinity_pred"}, inplace=True)
pred_df.head()

In [ ]:
df = pd.read_parquet(Path(path), )

In [5]:
import pyarrow.parquet as pq
path = Path("../output/data/combined_predictions_BindingDB.parquet")
df = pd.read_parquet(path)
# # Memory efficient approach for large files
# parquet_file = pq.ParquetFile(path)
# first_batch = next(parquet_file.iter_batches(batch_size=1000))
# df = first_batch.to_pandas()


In [6]:
df.head()

,row_index,Drug,BindingDB Ligand Name,Target Name,Target,Mutant,Kd (nM),Affinity,EC number,Target Class,...,Target_Class_Level_4,Target_Class_Level_5,Target_Class_Level_6,Drug_Features,Target_Features,Model,pred_affinity,Dataset,Split mode,Embedding
0,0,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,CHEMBL4209679,Iron-starvation protein PigA,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,Mutant,27000.0,4.568636,None,None,...,None,None,None,"[0.4142605662345886, 0.4228456914424896, 0.222...","[-0.024846425279974937, 0.31483280658721924, -...",GBM,5.712360,BindingDB,random,ESMv1
1,0,O=C1N(CCc2nnn[nH]2)C(=S)S\C1=C/c1ccccn1,CHEMBL4209679,Iron-starvation protein PigA,MDTLAPESTRQNLRSQRLNLLTNEPHQRLESLVKSKEPFASRDNFA...,Mutant,27000.0,4.568636,None,None,...,None,None,None,"[0.4142605662345886, 0.4228456914424896, 0.222...","[-0.024846425279974937, 0.31483280658721924, -...",XGBoost,5.183766,BindingDB,random,ESMv1
2,1,OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,"4-[[7-[2,6-bis(fluoranyl)phenyl]-9-chloranyl-5...",Ribosomal protein S6 kinase alpha-6,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,Mutant,10000.0,5.000000,2.7.11.1,Enzyme,...,AGC protein kinase group,AGC protein kinase RSK family,AGC protein kinase RSK subfamily,"[0.603754997253418, 0.49803397059440613, 0.045...","[-0.13943365216255188, -0.04169353097677231, -...",GBM,4.998585,BindingDB,random,ESMv1
3,1,OC(=O)c1ccc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(...,"4-[[7-[2,6-bis(fluoranyl)phenyl]-9-chloranyl-5...",Ribosomal protein S6 kinase alpha-6,MLPFAPQDEPWDREMEVFSGGGASSGEVNGLKMVDEPMEEGEADSC...,Mutant,10000.0,5.000000,2.7.11.1,Enzyme,...,AGC protein kinase group,AGC protein kinase RSK family,AGC protein kinase RSK subfamily,"[0.603754997253418, 0.49803397059440613, 0.045...","[-0.13943365216255188, -0.04169353097677231, -...",XGBoost,4.958881,BindingDB,random,ESMv1
4,2,CSc1cnc(N[C@H]2CC[C@@H](C2)Nc2ccc(cn2)-c2ccccc...,"US11248001, Example 467",Proprotein convertase subtilisin/kexin type 9,MGTVSSRRSWWPLPLLLLLLLLLGPAGARAQEDEDGDYEELVLALR...,Mutant,200.0,6.698970,3.4.21.-,Enzyme,...,Serine protease SB clan,Serine protease S8A subfamily,None,"[0.7361834645271301, 0.798686683177948, 0.7603...","[-0.02277339994907379, 0.23574493825435638, -0...",GBM,5.742586,BindingDB,random,ESMv1


In [37]:
df.columns

Index(['row_index', 'Drug', 'BindingDB Ligand Name', 'Target Name', 'Target',
       'Mutant', 'Kd (nM)', 'Affinity', 'EC number', 'Target Class',
       'FDA Approved', 'Target_Class_Level_1', 'Target_Class_Level_2',
       'Target_Class_Level_3', 'Target_Class_Level_4', 'Target_Class_Level_5',
       'Target_Class_Level_6', 'Drug_Features', 'Target_Features', 'Model',
       'pred_affinity', 'Dataset', 'Split mode', 'Embedding'],
      dtype='object')

In [7]:
df["Embedding"].unique()

array(['ESMv1', 'ESM2', 'MUTAPLM', 'ProteinCLIP'], dtype=object)

In [8]:

df["Split mode"].unique()

array(['random', 'cold_protein', 'cold_drug'], dtype=object)

In [10]:
df["Model"].unique()

array(['GBM', 'XGBoost'], dtype=object)

In [21]:
df.describe()

,row_index,Kd (nM),Affinity,pred_affinity
count,22696.000000,2.269600e+04,22696.000000,22696.000000
mean,502.599225,6.380074e+04,6.397350,6.626150
std,291.065784,8.705157e+05,1.469832,1.014957
min,0.000000,2.000000e-03,1.397940,2.004721
25%,249.000000,5.500000e+01,5.387216,5.995776
50%,512.000000,5.590000e+02,6.252588,6.649527
75%,756.000000,4.100000e+03,7.259637,7.216034
max,999.000000,4.000000e+07,11.698970,12.031569


In [31]:
top_k = 10
min_n = 5
res = metrics_per_category(
            df,
            "Target Class",
            by=["Dataset", "Split mode", "Embedding"],
            top_k=top_k,
            min_n=min_n,
        )

/Users/yvesgreatti/github/drug_discovery/src/ML_benchmark_prediction_analysis.py:148: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ]


In [34]:
res.head()

,Dataset,Split mode,Embedding,category,n,rmse,mae,mse,r2,pearson,median_ae,explained_variance
0,BindingDB,cold_drug,ProteinCLIP,Enzyme,1154.0,1.206324,0.926839,1.455216,0.014772,0.303518,0.752264,0.055100
1,BindingDB,cold_drug,ESM2,Enzyme,1154.0,1.235105,0.954617,1.525485,-0.032802,0.276508,0.799682,-0.006813
2,BindingDB,cold_drug,MUTAPLM,Enzyme,1154.0,1.270489,0.983193,1.614143,-0.092827,0.245591,0.804887,-0.041366
3,BindingDB,cold_drug,ESMv1,Enzyme,1154.0,1.296967,0.984334,1.682125,-0.138852,0.253811,0.796920,-0.123687
4,BindingDB,random,ESM2,Enzyme,924.0,1.149788,0.818582,1.322014,0.404883,0.641088,0.551421,0.406221


In [33]:
res.to_csv("../output/data/prediction_target_class_analysis.csv", index=False)

In [35]:
top_k = 10
min_n = 5
res = metrics_per_category(
            df,
            "Target Class",
            by=["Embedding"],
            top_k=top_k,
            min_n=min_n,
)
res.head()

/Users/yvesgreatti/github/drug_discovery/src/ML_benchmark_prediction_analysis.py:148: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ]


,Embedding,category,n,rmse,mae,mse,r2,pearson,median_ae,explained_variance
0,ProteinCLIP,Enzyme,2954.0,1.217115,0.911841,1.481370,0.173177,0.463002,0.706449,0.184095
1,ESM2,Enzyme,2954.0,1.226950,0.922839,1.505407,0.159761,0.455586,0.727113,0.171109
2,MUTAPLM,Enzyme,2954.0,1.253469,0.942858,1.571185,0.123047,0.436746,0.728664,0.142039
3,ESMv1,Enzyme,2954.0,1.272035,0.947503,1.618073,0.096876,0.426688,0.722131,0.097906
4,ESM2,GPCR,444.0,1.095626,0.874980,1.200397,-0.156156,0.347409,0.739559,0.053968


In [36]:
res.to_csv("../output/data/prediction_embedding_target_class_analysis.csv", index=False)

In [38]:
top_k = 10
min_n = 5
res = metrics_per_category(
            df,
            "Mutant",
            by=["Embedding"],
            top_k=top_k,
            min_n=min_n,
)
res.head()

/Users/yvesgreatti/github/drug_discovery/src/ML_benchmark_prediction_analysis.py:148: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ]


,Embedding,category,n,rmse,mae,mse,r2,pearson,median_ae,explained_variance
0,ESM2,Mutant,3288.0,1.257348,0.961553,1.580923,0.210568,0.496133,0.771457,0.227334
1,MUTAPLM,Mutant,3288.0,1.294957,0.993061,1.676913,0.162636,0.470563,0.797684,0.188594
2,ESMv1,Mutant,3288.0,1.383590,1.062740,1.914322,0.044086,0.444252,0.834271,0.066152
3,ProteinCLIP,Mutant,3288.0,1.397452,1.065936,1.952872,0.024836,0.397982,0.834713,0.081640
4,ESM2,No match,1778.0,1.398066,1.056359,1.954588,0.277872,0.551784,0.829176,0.289326


In [39]:
res.to_csv("../output/data/prediction_embedding_mutant_analysis.csv", index=False)

In [ ]:
# Create a simple scatter plot of predictions vs true values
plt.figure(figsize=(10, 8))

# Plot by embedding type
embeddings = df['Embedding'].unique()
colors = plt.cm.Set1(np.linspace(0, 1, len(embeddings)))

for i, embedding in enumerate(embeddings):
    mask = df['Embedding'] == embedding
    plt.scatter(df[mask]['Affinity'], df[mask]['affinity_pred'], 
               alpha=0.6, label=embedding, color=colors[i], s=20)

# Add perfect prediction line
min_val = min(df['Affinity'].min(), df['affinity_pred'].min())
max_val = max(df['Affinity'].max(), df['affinity_pred'].max())
plt.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.8, label='Perfect Prediction')

plt.xlabel('True Affinity', fontsize=12)
plt.ylabel('Predicted Affinity', fontsize=12)
plt.title('Predicted vs True Affinity Values (1000 samples)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()